## 1. Importaciones y Configuración

Importamos las bibliotecas necesarias:
- **asyncio**: Para ejecución asíncrona
- **Annotated y Field**: Para anotaciones de tipo enriquecidas
- **AzureOpenAIChatClient**: Cliente para Azure OpenAI
- **AzureCliCredential**: Autenticación con credenciales de Azure CLI

In [ ]:
import asyncio
from typing import Annotated
from pydantic import Field
from azure.identity import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient
from dotenv import load_dotenv
import os

load_dotenv()

print("✅ Importaciones completadas")

## 2. Definición de la Función Tool

### La Función `get_weather`

Esta función simula una API de clima. En un caso real, llamaría a un servicio como OpenWeatherMap.

#### Características Importantes:

1. **Annotated Type**: `Annotated[str, Field(description=...)]`
   - El primer argumento es el tipo (`str`)
   - El segundo es un `Field` de Pydantic con descripción
   - Esta descripción ayuda al LLM a entender cuándo y cómo usar la función

2. **Docstring**: Proporciona contexto adicional sobre el propósito de la función

3. **Type Hints**: Permiten que el framework valide automáticamente los argumentos

El LLM lee la descripción y decide automáticamente llamar a esta función cuando el usuario pregunta sobre el clima.

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="La ubicación para obtener el clima.")],
) -> str:
    """Obtiene el clima para una ubicación dada."""
    return f"El clima en {location} está nublado con una máxima de 15°C."

print("✅ Función de clima definida")

# Probar la función directamente
print("\n🧪 Prueba directa de la función:")
print(get_weather("Madrid"))

## 3. Creación del Cliente de Azure OpenAI

### Configuración del Cliente

El `AzureOpenAIChatClient` requiere:
- **credential**: Autenticación (usamos Azure CLI)
- **endpoint**: URL del servicio Azure OpenAI
- **deployment_name**: Nombre del modelo desplegado (ej: gpt-4, gpt-4o)

Las variables se leen del archivo `.env` para mantener la configuración segura y separada del código.

In [ ]:
openai_client = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("MODEL"),
)

print("✅ Cliente de Azure OpenAI creado")
print(f"📍 Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"🤖 Modelo: {os.getenv('MODEL')}")

## 4. Creación del Agente Especializado (Weather Agent)

### WeatherAgent

Este es un **agente especializado** que:
- Tiene un propósito único: responder preguntas sobre el clima
- Tiene acceso a la función `get_weather` como herramienta
- Puede ser usado como **tool** por otros agentes

#### Parámetros Importantes:
- **name**: Identificador del agente
- **description**: Ayuda al agente principal a decidir cuándo usar este agente
- **instructions**: Guía el comportamiento del agente
- **tools**: Lista de funciones/herramientas disponibles

El agente puede decidir cuándo necesita llamar a `get_weather` basándose en la consulta del usuario.

In [ ]:
weather_agent = openai_client.create_agent(
    name="WeatherAgent",
    description="Un agente que responde preguntas sobre el clima.",
    instructions="Respondes preguntas sobre el clima.",
    tools=get_weather
)

print("✅ Weather Agent creado")
print("🛠️  Herramientas disponibles: get_weather")

## 5. Creación del Agente Principal (Main Agent)

### Main Agent con Composición

Este es un patrón poderoso de **composición de agentes**:

1. El agente principal tiene sus propias instrucciones (responder en francés)
2. Puede usar al `weather_agent` como una herramienta mediante `.as_tool()`
3. Decide automáticamente cuándo delegar al agente especializado

### Flujo de Ejecución:
```
1. Usuario pregunta: "¿Qué clima hace en Amsterdam?"
2. Main Agent detecta que necesita información del clima
3. Main Agent llama a weather_agent.as_tool()
4. Weather Agent llama a get_weather("Amsterdam")
5. Weather Agent retorna resultado al Main Agent
6. Main Agent traduce la respuesta al francés
7. Usuario recibe respuesta en francés
```

Esta arquitectura permite:
- **Reutilización**: El mismo weather_agent puede usarse en múltiples contextos
- **Especialización**: Cada agente se enfoca en una tarea
- **Escalabilidad**: Fácil agregar más agentes especializados

In [ ]:
main_agent = openai_client.create_agent(
    instructions="Eres un asistente útil que responde en francés.",
    tools=weather_agent.as_tool()
)

print("✅ Main Agent creado")
print("🇫🇷 Idioma de respuesta: Francés")
print("🛠️  Herramientas disponibles: WeatherAgent (como tool)")

## 6. Ejecución del Agente

### Proceso de Ejecución:

1. Se envía una consulta al `main_agent`
2. El agente analiza la consulta y determina que necesita información del clima
3. Automáticamente invoca al `weather_agent` como herramienta
4. El `weather_agent` ejecuta `get_weather("Amsterdam")`
5. La respuesta se propaga de vuelta al `main_agent`
6. El `main_agent` formula la respuesta final en francés

Todo este proceso es **automático** - no necesitamos programar explícitamente la lógica de decisión.

In [ ]:
async def main():
    print("🚀 Ejecutando consulta sobre el clima...\n")
    print("❓ Pregunta: What is the weather like in Amsterdam?\n")
    
    result = await main_agent.run("What is the weather like in Amsterdam?")
    
    print("💬 Respuesta del agente:")
    print("="*60)
    print(result.text)
    print("="*60)
    print("\n✅ Nota: La respuesta está en francés según las instrucciones del agente principal")

# Ejecutar la función asíncrona
await main()

## 7. Experimentos Adicionales

Probemos diferentes consultas para entender mejor cómo funcionan los agentes:

In [ ]:
# Experimento 1: Múltiples ubicaciones
async def test_multiple_locations():
    print("🧪 Experimento 1: Múltiples ubicaciones\n")
    result = await main_agent.run("Compare the weather in Paris, London, and Berlin")
    print(result.text)
    print()

await test_multiple_locations()

In [ ]:
# Experimento 2: Pregunta sin relación con el clima
async def test_non_weather_query():
    print("🧪 Experimento 2: Pregunta sin relación con el clima\n")
    result = await main_agent.run("What is the capital of Spain?")
    print(result.text)
    print("\n📝 Observa que el agente responde en francés pero NO llama a la función del clima")
    print()

await test_non_weather_query()

## 8. Análisis y Conclusiones

### Ventajas del Patrón Function Agent:

1. **Automatización Inteligente**:
   - El LLM decide cuándo llamar funciones sin lógica explícita
   - Reduce código boilerplate significativamente

2. **Type Safety**:
   - Las anotaciones de tipo previenen errores
   - Pydantic valida automáticamente los argumentos

3. **Composición Modular**:
   - Agentes pueden usar otros agentes como herramientas
   - Arquitectura escalable y mantenible

4. **Separación de Responsabilidades**:
   - Cada agente tiene un propósito claro
   - Fácil testing y debugging

### Casos de Uso Prácticos:

1. **Asistentes Virtuales**:
   - Agente principal que coordina múltiples servicios
   - Agentes especializados para clima, calendario, email, etc.

2. **Automatización Empresarial**:
   - Consultar bases de datos
   - Interactuar con APIs internas
   - Generar reportes

3. **Análisis de Datos**:
   - Agentes que ejecutan consultas SQL
   - Generación de gráficos
   - Cálculos estadísticos

4. **Integración de Servicios**:
   - Conectar con APIs externas (Stripe, Twilio, etc.)
   - Orchestrar microservicios
   - Workflows complejos

### Mejores Prácticas:

1. **Descripciones Claras**: Escribe descripciones detalladas en Field() para ayudar al LLM
2. **Validación de Inputs**: Usa tipos Pydantic para validación robusta
3. **Manejo de Errores**: Implementa try-catch en funciones para respuestas graceful
4. **Testing**: Prueba funciones independientemente antes de integrarlas
5. **Logging**: Registra llamadas a funciones para debugging y auditoría

### Extensiones Posibles:

1. **API Real de Clima**: Integrar con OpenWeatherMap o WeatherAPI
2. **Caché**: Guardar resultados para evitar llamadas repetidas
3. **Múltiples Idiomas**: Soportar respuestas en varios idiomas
4. **Historial**: Mantener contexto de conversaciones previas
5. **Rate Limiting**: Controlar frecuencia de llamadas a APIs externas